# From data to decisions: Understanding movie success through rotten tomatoes and TMDB

**Author:** David Mwai Gathimba

## Project Overview

In this project, we are going to provide a data-driven report to Microsoft's new movie studio on the success factors of movies using data from Rotten Tomatoes and TMDB. One of the basic business questions for Microsoft is figuring out which types of movies are likely to do the best in theaters—since the company has virtually no experience in the world of making motion pictures. To look for the trends and correlations, we use datasets of movie reviews, movie ratings, and movie popularity metrics. We walk through data cleaning, exploratory data analysis, and visualization to expose leading indicators. The results also show that success is associated with key factors including genre, review scores, and popularity. According to the results, we suggest that Microsoft should make movies with good ratings and movements in popular genres to get the highest possible performance in the competitive movie circle.


## Business Problem
***
Microsoft is in the process of launching a new film studio, entering into the competitive film industry, which they have no experience with, thus the reason they are leveraging data to do so. The business problem we are trying to solve is to find out the most important features which influence the revenue and popularity of movies. In this post, we will answer a couple of data questions on the observation: What types of movies are doing well at the box office scene genre-wise? Do Rotten Tomatoes film scores affect box office revenue? How does the popularity of a movie on TMDB relate to its box-office performance? What difference would it make to a movie whether these attributes like run time or period of a release play a vital role in determining the success of a movie? So what about those insights for Microsoft, to inform their entry into movie production in a strategic way?

These queries are designed to directly correlate to quantitative properties of movies, and are looking for patterns or trends the data might reveal, to advise Microsoft at the strategy level. Having insight into these factors is essential for Microsoft to mitigate costs, allocate resources efficiently, and ensure its movie studio has the best chance of success in an already crowded market. In addition to genre popularity and the effect of reviews and popularity, we will use 10-fold cross-validation to foresee potential box office performance that could act as a guide star for Microsoft, which tries to be smarter about producing data-driven production decisions.
***

## Data Understanding
***
The data used in this project comes from two primary sources: Rotten Tomatoes (rt.reviews.tsv) and TheMovieDB (tmdb.movies.csv). These datasets provide a comprehensive view of movie reviews, ratings, and popularity, which are essential for analyzing the factors that contribute to a movie's success.

The Rotten Tomatoes dataset includes reviews and ratings for a wide range of movies. This data is crucial for understanding how critical reception correlates with box office performance. Key variables in this dataset include the movie title, review scores, and the number of reviews. The sample consists of various movies reviewed on the Rotten Tomatoes platform, capturing both popular and niche films.

The TMDB dataset contains detailed information about movies listed on TheMovieDB, including attributes such as movie title, genre, popularity scores, release dates, and runtime. This dataset helps us analyze how movie characteristics and their popularity on a major database influence their success. The key variables here include the movie title, genre, popularity score, release date, and runtime. The sample encompasses a diverse array of movies from different genres and periods.

Our target variable in this analysis is the box office revenue, which, although not directly included in the provided datasets, can be inferred or matched from external sources if needed. For this project, we focus on understanding how the review scores from Rotten Tomatoes and the popularity scores from TMDB influence a movie's box office success.

The properties of the variables we intend to use include:
- *Review Scores (Rotten Tomatoes):* Represents the average rating given by critics or users. This variable is numerical and ranges from 0 to 100.
- *Popularity Scores (TMDB):* Indicates the popularity of a movie on TheMovieDB platform. This is a numerical variable with values reflecting the relative popularity of the movies.
- *Genres:* Categorical variable representing the movie genre (e.g., Action, Drama, Comedy).
- *Release Dates:* Temporal variable indicating when the movie was released.
- *Runtime:* Numerical variable representing the duration of the movie in minutes.

These variables provide a robust basis for exploring the correlations and trends that can inform Microsoft's movie production strategy.
***

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# Loading the data
# Load Rotten Tomatoes reviews data
df_rt = pd.read_csv('Data/rt.reviews.tsv', delimiter='\t',  encoding='latin-1')

# Load TMDB movies data
df_tmdb = pd.read_csv('Data/tmdb.movies.csv')

# First few rows of each
print(df_rt.head())
print('***')
print(df_tmdb.head())

   id                                             review rating   fresh  \
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   
3   3  Continuing along a line introduced in last yea...    NaN   fresh   
4   3             ... a perverse twist on neorealism...     NaN   fresh   

           critic  top_critic         publisher               date  
0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0           io9.com       May 23, 2018  
2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
3   Daniel Kasman           0              MUBI  November 16, 2017  
4             NaN           0      Cinema Scope   October 12, 2017  
***
   Unnamed: 0            genre_ids     id original_language  \
0           0      [12, 14, 10751]  12444                en   
1    

## Data Preparation
I dropped the colums that were not useful in the analysis of movie perfomance. I also created some variables which were provided as text and others which provide a more comprehensive measure of a movie's success. Rows with critical values were dropped to eliminate biases. I identified the outliers and handled them to prevent them from skewing the results. I merged the datasets to comprehensive dataset for analysis.


In [14]:
# check for missing values in tmdb_movies
print(df_tmdb.isnull().sum)

<bound method DataFrame.sum of        Unnamed: 0  genre_ids     id  original_language  original_title  \
0           False      False  False              False           False   
1           False      False  False              False           False   
2           False      False  False              False           False   
3           False      False  False              False           False   
4           False      False  False              False           False   
...           ...        ...    ...                ...             ...   
26512       False      False  False              False           False   
26513       False      False  False              False           False   
26514       False      False  False              False           False   
26515       False      False  False              False           False   
26516       False      False  False              False           False   

       popularity  release_date  title  vote_average  vote_count  
0           F

In [18]:
# drop rows with missing values in tmdb movies
df_tmdb = df_tmdb.dropna()

In [19]:
print(df_tmdb.isnull().sum)

<bound method DataFrame.sum of        Unnamed: 0  genre_ids     id  original_language  original_title  \
0           False      False  False              False           False   
1           False      False  False              False           False   
2           False      False  False              False           False   
3           False      False  False              False           False   
4           False      False  False              False           False   
...           ...        ...    ...                ...             ...   
26512       False      False  False              False           False   
26513       False      False  False              False           False   
26514       False      False  False              False           False   
26515       False      False  False              False           False   
26516       False      False  False              False           False   

       popularity  release_date  title  vote_average  vote_count  
0           F

In [20]:
# check for missing values in Rotten Tomatoes reviews
print(df_rt.isnull().sum)

<bound method DataFrame.sum of           id  review  rating  fresh  critic  top_critic  publisher   date
0      False   False   False  False   False       False      False  False
1      False   False    True  False   False       False      False  False
2      False   False    True  False   False       False      False  False
3      False   False    True  False   False       False      False  False
4      False   False    True  False    True       False      False  False
...      ...     ...     ...    ...     ...         ...        ...    ...
54427  False   False    True  False   False       False      False  False
54428  False    True   False  False   False       False      False  False
54429  False    True   False  False   False       False      False  False
54430  False    True   False  False   False       False      False  False
54431  False    True   False  False   False       False      False  False

[54432 rows x 8 columns]>


In [21]:
# drop rows with missing values in Rotten Tomatoes reviews
df_rt = df_rt.dropna()

In [22]:
print(df_rt)

         id                                             review rating   fresh  \
0         3  A distinctly gallows take on contemporary fina...    3/5   fresh   
6         3  Quickly grows repetitive and tiresome, meander...      C  rotten   
7         3  Cronenberg is not a director to be daunted by ...    2/5  rotten   
11        3  While not one of Cronenberg's stronger films, ...     B-   fresh   
12        3  Robert Pattinson works mighty hard to make Cos...    2/4  rotten   
...     ...                                                ...    ...     ...   
54419  2000            Sleek, shallow, but frequently amusing.  2.5/4   fresh   
54420  2000  The spaniel-eyed Jean Reno infuses Hubert with...    3/4   fresh   
54421  2000  Manages to be somewhat well-acted, not badly a...  1.5/4  rotten   
54422  2000  Arguably the best script that Besson has writt...  3.5/5   fresh   
54424  2000  Dawdles and drags when it should pop; it doesn...  1.5/5  rotten   

               critic  top_